In [2]:
import json
import numpy as np
import sys

def printf(format, *args):
    sys.stdout.write(format % args)
    
def load_json():
    
    authors = []
    features = []
    scores = []
    
    with open('dataset.json') as data_file:
        data = json.load(data_file)
        labels = data["column_descriptors"]
        author_data = data["author_data"]
        
        for author in author_data.keys():
            authors.append(author)
            features.append(author_data[author]['feature_vecs'])
            scores_str = author_data[author]['scores']
            scores.append(np.asarray([int(score_) for score_ in scores_str]))
    
    features = [np.asarray(feature) for feature in features]
    
    return labels, authors, features, scores

In [3]:
labels, authors, features, scores = load_json()
print (authors[0], features[0].shape, scores[0].shape, "\n")
print (labels)

(u'bodmas', (3, 34), (3,), '\n')
[u'number_of_modules', u'lines_of_code', u'lines_of_code_per_module', u'McCabes_cyclomatic_complexity', u'McCabes_cyclomatic_complexity_per_module', u'lines_of_comment', u'lines_of_comment_per_module', u'lines_of_code_per_line_of_comment', u'McCabes_cyclomatic_complexity_per_line_of_comment', u'IF4', u'IF4_per_module', u'IF4_visible', u'IF4_visible_per_module', u'IF4_concrete', u'IF4_concrete', u'rejected_lines_of_code\n', u'Files', u'Lines', u'AVG Len', u'Code', u'Comments', u'White SP', u'Cd/Cm+WS', u'Cd/Cm', u'Cd/WS', u'% Code', u'cnt_classes', u'max_member_funs', u'max_nested_loops', u'max_nesting_depth', u'max_params_in_decl', u'member_funs', u'member_vars', u'min_member_funs\n']


In [4]:
# y is vector of labels
def create_labels(y):
    y_l = np.copy(y)
    for i in range(y.shape[0]):
        if y[i] >= 80: 
            y_l[i] = 5
        if y[i] >= 60 and y[i] < 80: 
            y_l[i] = 4
        if y[i] >= 40 and y[i] < 60:
            y_l[i] = 3
        if y[i] >= 20 and y[i] < 40:
            y_l[i] = 2
        if y[i] < 20: 
            y_l[i] = 1
    return y_l

In [5]:
# training and test split
import random
def createTrainTestSplit():
    cnt_train = int(0.8 * len(authors)) + 1

    train_authors_indices = set()

    while len(train_authors_indices) < cnt_train:
        train_authors_indices.add(random.randint(0, len(authors) - 1))

    test_authors_indices = set()

    for i in range(len(authors)):
        if i not in train_authors_indices:
            test_authors_indices.add(i)

    train_authors = [authors[i] for i in train_authors_indices]

    test_authors = [authors[i] for i in test_authors_indices]
    
    X_train_list = []
    y_train_list = []
    X_test_list = []
    y_test_list = []

    for index in train_authors_indices:
        X_train_list.extend(features[index])
        y_train_list.extend(scores[index])

    for index in test_authors_indices:
        X_test_list.extend(features[index])
        y_test_list.extend(scores[index])

    X_train = np.asarray(X_train_list)
    y_train = np.asarray(y_train_list)
    X_test = np.asarray(X_test_list)
    y_test = np.asarray(y_test_list)
    
    y_train = create_labels(y_train) 
    y_test = create_labels(y_test)
    
    return X_train, y_train, X_test, y_test

In [6]:
X_train, y_train, X_test, y_test = createTrainTestSplit()
print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((15309, 34), (15309,), (3358, 34), (3358,))


In [6]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

print (clf.score(X_test, y_test))

0.264815344976


In [7]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
print (abc.score(X_test, y_test))


0.311193816204


In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
print (gbc.score(X_test, y_test))

0.365588319496


In [22]:
from sklearn.neighbors import KNeighborsClassifier

for n in range(1, 11):
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X_train, y_train)
    print ('neighbors', n, "score", neigh.score(X_test, y_test))

('neighbors', 1, 'score', 0.6587254318046456)
('neighbors', 2, 'score', 0.60720667063728406)
('neighbors', 3, 'score', 0.53782013103037518)
('neighbors', 4, 'score', 0.41542584871947585)
('neighbors', 5, 'score', 0.35973793924955333)
('neighbors', 6, 'score', 0.34276354973198331)
('neighbors', 7, 'score', 0.32638475282906493)
('neighbors', 8, 'score', 0.32132221560452651)
('neighbors', 9, 'score', 0.32995830851697439)
('neighbors', 10, 'score', 0.33204288266825494)


In [10]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=30)
etc.fit(X_train, y_train)
print (etc.score(X_test, y_test))

0.723733180647


In [7]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train, y_train)
print (dtc.score(X_test, y_test))

0.662298987493


In [11]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)

In [12]:
tree_to_code(dtc, labels)

def tree(number_of_modules, lines_of_code, lines_of_code_per_module, McCabes_cyclomatic_complexity, McCabes_cyclomatic_complexity_per_module, lines_of_comment, lines_of_comment_per_module, lines_of_code_per_line_of_comment, McCabes_cyclomatic_complexity_per_line_of_comment, IF4, IF4_per_module, IF4_visible, IF4_visible_per_module, IF4_concrete, IF4_concrete, rejected_lines_of_code
, Files, Lines, AVG Len, Code, Comments, White SP, Cd/Cm+WS, Cd/Cm, Cd/WS, % Code, cnt_classes, max_member_funs, max_nested_loops, max_nesting_depth, max_params_in_decl, member_funs, member_vars, min_member_funs
):
  if rejected_lines_of_code
 <= 1.5:
    if McCabes_cyclomatic_complexity_per_module <= 36.5:
      if lines_of_code <= 20.5:
        if Comments <= 3.5:
          if % Code <= 0.8949999809265137:
            if max_params_in_decl <= 0.5:
              if White SP <= 4.5:
                if AVG Len <= 26.0:
                  if Cd/Cm <= 10.75:
                    if max_nested_loops <= 1.5:
       

In [ ]:
from sklearn.neural_network import MLPClassifier

alphas = [0.0001, 0.001, 0.01, 0.1]
solvers = ["adam", "lbfgs", "sgd"]
learning_rates = ["constant", "adaptive", "invscaling"]

for alpha in alphas:
    print("Alpha:\t", alpha)
    for solver in solvers:
        printf("Solver:\t%s\n", solver)
        for learning_rate in learning_rates:
            for i in range(0, 5):
                X_train, y_train, X_test, y_test = createTrainTestSplit()
                printf("Train/test split number %d\n", i)
                printf("\tLearning rate:\t%s\n", learning_rate)
                mlpC = MLPClassifier(alpha=alpha, batch_size='auto', learning_rate=learning_rate, learning_rate_init=0.01, power_t=0.5, shuffle=True, max_iter=500)
                mlpC.fit(X_train, y_train)
                printf("\tMLPC Score:\t%f\n\n", mlpC.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

criterions = [ "gini", "entropy"]
estimators_size = [50, 100, 150, 200]

for criterion in criterions:
    for n_estimators in estimators_size:
        values = []
        printf("RFC\t%d estimators\t%s criterion\n", n_estimators, criterion)
        for i in range(0, 5):
            X_train, y_train, X_test, y_test = createTrainTestSplit()
            rfc = RandomForestClassifier(n_estimators=n_estimators, max_features='log2', criterion=criterion)
            rfc.fit(X_train, y_train)
            values.append(rfc.score(X_test, y_test))
        
        values.sort()
        print("Median:", values[2])
        print("Average:", sum(values) / 5)
        print("Min:", min(values))
        print("Max:", max(values))

In [19]:
from sklearn import tree
export_dtc = tree.DecisionTreeClassifier(max_depth=25)
export_dtc.fit(X_train, y_train)
print (export_dtc.score(X_test, y_test))

0.653365098273


In [20]:
import sklearn.tree

tree.export_graphviz(dtc, 'tree.dot')